# Make allometric equations mask

Following Harris et al 2021
([paper](https://www.nature.com/articles/s41558-020-00976-6),
[spreadsheet](https://docs.google.com/spreadsheets/d/1Hb67l3xYCfgxKu9TnpbEfY6iQo4ISNEvQewpmYvH9yQ/edit#gid=1620488341)),
and Farina et al
([paper](https://docs.google.com/document/d/1qoIoYBghr7FfqZlcT8h5BGMww_Obtnrw/edit)),
we want to use the following datasets to determine which allometric equations to
use:

- Ecoregions2017. Dinerstein, Eric, David Olson, Anup Joshi, Carly Vynne, Neil D
  Burgess, Eric Wikramanayake, Nathan Hahn, et al. 2017. “An Ecoregion-Based
  Approach to Protecting Half the Terrestrial Realm.” BioScience 67 (6): 534–45.
  https://doi.org/10.1093/biosci/bix014. Retrieved from
  https://ecoregions2017.appspot.com/ on Mar 5th, 2021.

- NLCD. Retrieved from CarbonPlan data storage on GCP.

- EOSD.

- IGBP. Friedl, M.A., A.H. Strahler, and J. Hodges. 2010. ISLSCP II MODIS
  (Collection 4) IGPB Land Cover, 2000-2001. In Hall, Forrest G., G. Collatz, B.
  Meeson, S. Los, E. Brown de Colstoun, and D. Landis (eds.). ISLSCP Initiative
  II Collection. Data set. Available on-line [http://daac.ornl.gov/] from Oak
  Ridge National Laboratory Distributed Active Archive Center, Oak Ridge,
  Tennessee, U.S.A. doi:10.3334/ORNLDAAC/968. Retrieved from
  https://daac.ornl.gov/daacdata/islscp_ii/vegetation/modis_landcover_xdeg/data/.
  Documented in
  https://daac.ornl.gov/daacdata/islscp_ii/vegetation/modis_landcover_xdeg/comp/1_modis_landcover_doc.pdf.

In this notebook, we load in each dataset, transform everything to the target
grid, and store the output.


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import fsspec
import os
import shutil
import regionmask
import rioxarray

from itertools import product
from zarr.errors import GroupNotFoundError

from carbonplan_trace.v1.utils import save_to_zarr

from gcsfs import GCSFileSystem

fs = GCSFileSystem(cache_timeout=0)

In [ ]:
def get_tile_in_xr(path):
    mapper = fsspec.get_mapper(path)
    try:
        ds = xr.open_zarr(mapper, chunks=None)
        ds.attrs["crs"] = "EPSG:4326"

        return ds
    except GroupNotFoundError:
        print(f"{path} empty, skipping")


def convert_gdf_into_tiles(tile_ds, gdf, value_col, value_name):
    # get coordinates of target tile
    lon_res = tile_ds.lon.values[1] - tile_ds.lon.values[0]
    lon = np.arange(
        tile_ds.lon.values[0], tile_ds.lon.values[-1] + (lon_res / 2), lon_res
    )
    lat_res = tile_ds.lat.values[1] - tile_ds.lat.values[0]
    lat = np.arange(
        tile_ds.lat.values[0], tile_ds.lat.values[-1] + (lat_res / 2), lat_res
    )

    # turn gdf into xarray
    output_da = regionmask.mask_geopandas(
        gdf, numbers=value_col, lon_or_obj=lon, lat=lat
    )
    output_da.name = value_name

    return output_da


def convert_raster_into_tiles(tile_ds, raster):
    output = raster.rio.reproject_match(tile_ds)

    return output

# Find target tiles


In [ ]:
# Target tiles
lat_tags = [
    "80N",
    "70N",
    "60N",
    "50N",
    "40N",
    "30N",
    "20N",
    "10N",
    "00N",
    "10S",
    "20S",
    "30S",
    "40S",
    "50S",
]
lon_tags = [f"{n:03}W" for n in np.arange(10, 190, 10)] + [
    f"{n:03}E" for n in np.arange(0, 180, 10)
]

tile_paths = []
for lat, lon in list(product(lat_tags, lon_tags)):
    tile_paths.append(f"gs://carbonplan-climatetrace/v0/tiles/{lat}_{lon}.zarr")

In [ ]:
len(lat_tags) * len(lon_tags)

In [ ]:
# try reading each tile and remove tile name if empty
empty_tiles = []
for tp in tile_paths:
    target_tile = get_tile_in_xr(tp)
    if not target_tile:
        empty_tiles.append(tp)

for tp in empty_tiles:
    tile_paths.remove(tp)

In [ ]:
tile_paths

# Read in each dataset, consolidate as appropriate, then turn into an raster in the same format as target tiles


### Ecoregions


In [ ]:
fp = "gs://carbonplan-scratch/trace_scratch/Ecoregions2017/Ecoregions2017.shp"
ecoregions = gpd.read_file(fp)
ecoregions.head()

In [ ]:
for tp in tile_paths:
    # use the same filename as target tiles for output
    fn = tp.split("/")[-1]
    output_path = f"gs://carbonplan-scratch/trace_scratch/ecoregions_mask/{fn}"

    if fs.exists(output_path):
        print(f"Skipping {fn}")
        pass
    else:
        print(f"Processing {fn}")
        # read in the target tile
        target_tile = get_tile_in_xr(tp)

        # convert ecoregions shapefile into target tile format
        output_da = convert_gdf_into_tiles(
            tile_ds=target_tile,
            gdf=ecoregions,
            value_col="ECO_ID",
            value_name="ecoregion",
        )

        # save the output
        save_to_zarr(ds=output_da.to_dataset(), url=output_path)

In [ ]:
# load a tile to double check output
ds = get_tile_in_xr(
    f"gs://carbonplan-scratch/trace_scratch/ecoregions_mask/50N_130W.zarr"
)

In [ ]:
ds

In [ ]:
ds.ecoregion.isnull().sum().values

In [ ]:
ds.ecoregion[::100, ::100].plot()

### NLCD


In [ ]:
nlcd_conus = xr.open_rasterio(
    "gs://carbonplan-data/raw/nlcd/conus/30m/2001.tif", parse_coordinates=True
)

In [ ]:
nlcd_conus[::100, ::100].plot()

In [ ]:
# look at the bounding box of NLCD data

from pyproj import Transformer

transformer = Transformer.from_crs(nlcd_conus.crs, "EPSG:4326")

lat, lon = transformer.transform(nlcd_conus.x.values[0], nlcd_conus.y.values[0])
lat, lon

In [ ]:
lat, lon = transformer.transform(
    nlcd_conus.x.values[-1], nlcd_conus.y.values[-1]
)
lat, lon

In [ ]:
lons_of_interest = ["130W", "120W", "110W", "100W", "090W", "080W"]
lats_of_interest = ["50N", "40N", "30N"]

for lat, lon in list(product(lats_of_interest, lons_of_interest)):
    # use the same filename as target tiles for output
    fn = f"{lat}_{lon}.zarr"
    #     output_path = f'gs://carbonplan-scratch/trace_scratch/nlcd_cache/{fn}'
    output_path = f"gs://carbonplan-scratch/trace_scratch/ecoregions_mask/{fn}"

    if not fs.exists(output_path):
        # if the target tile doesn't exist, then pass
        print(f"Skipping {fn}, file does not exist")
        pass

    elif fs.exists(output_path + "/nlcd/"):
        # if we have already process this tile, also pass
        print(f"Skipping {fn}, NLCD data already present")
        pass

    else:
        # otherwise reproject the tile
        print(f"Processing {fn}")
        # read in the target tile
        target_tile = get_tile_in_xr(output_path)
        target_tile = target_tile.rename(lon="x", lat="y")

        # convert NLCD raster into target tile format
        output_da = convert_raster_into_tiles(
            tile_ds=target_tile, raster=nlcd_conus
        )
        output_da = output_da.drop_vars("spatial_ref")
        output_da = output_da.squeeze(dim="band", drop=True)
        output_da.attrs = {"crs": "EPSG:4326"}
        output_da.coords["x"] = target_tile.x
        output_da.coords["y"] = target_tile.y

        target_tile["nlcd"] = output_da
        target_tile = target_tile.rename(x="lon", y="lat")

        # save the output
        save_to_zarr(
            ds=target_tile,
            url=output_path,
            list_of_variables=["nlcd"],
            mode="a",
        )

In [ ]:
# load a tile to double check output
ds = get_tile_in_xr(
    f"gs://carbonplan-scratch/trace_scratch/ecoregions_mask/50N_130W.zarr"
)
ds

In [ ]:
ds.nlcd[::100, ::100].plot()

In [ ]:
np.unique(ds.nlcd.values)

In [ ]:
ds.nlcd.isnull().sum().values

In [ ]:
ds.ecoregion.isnull().sum().values

In [ ]:
ds.ecoregion[::100, ::100].plot()

In [ ]:
nlcd_ak = xr.open_rasterio(
    "https://storage.googleapis.com/carbonplan-data/raw/nlcd/ak/30m/2011.tif",
    parse_coordinates=True,
)

In [ ]:
nlcd_ak[::100, ::100].plot()

In [ ]:
# look at the bounding box of NLCD data

from pyproj import Transformer

transformer = Transformer.from_crs(nlcd_ak.crs, "EPSG:4326")

lat, lon = transformer.transform(nlcd_ak.x.values[0], nlcd_ak.y.values[0])
lat, lon

In [ ]:
lat, lon = transformer.transform(nlcd_ak.x.values[-1], nlcd_ak.y.values[-1])
lat, lon

In [ ]:
lons_of_interest = [
    "150E",
    "160E",
    "170E",
    "180W",
    "170W",
    "160W",
    "150W",
    "140W",
]
lats_of_interest = ["70N", "60N"]

for lat, lon in list(product(lats_of_interest, lons_of_interest)):
    # use the same filename as target tiles for output
    fn = f"{lat}_{lon}.zarr"
    output_path = f"gs://carbonplan-scratch/trace_scratch/ecoregions_mask/{fn}"

    if not fs.exists(output_path):
        # if the target tile doesn't exist, then pass
        print(f"Skipping {fn}, file does not exist")
        pass

    elif fs.exists(output_path + "/nlcd/"):
        # if we have already process this tile, also pass
        print(f"Skipping {fn}, NLCD data already present")
        pass

    else:
        # otherwise reproject the tile
        print(f"Processing {fn}")
        # read in the target tile
        target_tile = get_tile_in_xr(output_path)
        target_tile = target_tile.rename(lon="x", lat="y")

        # convert NLCD raster into target tile format
        output_da = convert_raster_into_tiles(
            tile_ds=target_tile, raster=nlcd_ak
        )
        output_da = output_da.drop_vars("spatial_ref")
        output_da = output_da.squeeze(dim="band", drop=True)
        output_da.attrs = {"crs": "EPSG:4326"}
        output_da.coords["x"] = target_tile.x
        output_da.coords["y"] = target_tile.y

        target_tile["nlcd"] = output_da
        target_tile = target_tile.rename(x="lon", y="lat")

        # save the output
        save_to_zarr(
            ds=target_tile,
            url=output_path,
            list_of_variables=["nlcd"],
            mode="a",
        )

In [ ]:
# load a tile to double check output
ds = get_tile_in_xr(
    f"gs://carbonplan-scratch/trace_scratch/ecoregions_mask/70N_140W.zarr"
)
ds

In [ ]:
np.unique(ds.nlcd.values)

In [ ]:
ds.nlcd[::100, ::100].plot()

In [ ]:
ds.ecoregion[::100, ::100].plot()

### IGBP

documentations:
https://daac.ornl.gov/daacdata/islscp_ii/vegetation/modis_landcover_xdeg/comp/


In [ ]:
# fn = '../data/IGBP/modis_landcover_class_qd.asc'
# # the first 6 lines are additional info not data
# headers = 6
# igbp = np.genfromtxt(fn, skip_header=headers)

In [ ]:
# with open(fn) as f:
#     head = [next(f) for x in range(headers)]
# print(head)

In [ ]:
# from rasterio.transform import Affine

# # use info in the headers
# ncols = 1440
# nrows = 720
# xll = -180
# yll = -90
# res = .25

# transform = Affine.translation(xll, yll+res*nrows) * Affine.scale(res, -res)
# transform

In [ ]:
# import rasterio
# import xarray as xr

# fn = 'IGBP.tif'
# local_target = f"../data/{fn}"
# remote_target = f"gs://carbonplan-scratch/trace_scratch/{fn}"

# os.remove(local_target)
# with rasterio.open(
#     local_target,
#     'w',
#     driver='GTiff',
#     height=igbp.shape[0],
#     width=igbp.shape[1],
#     count=1,
#     dtype=igbp.dtype,
#     crs='+proj=latlong',
#     transform=transform,
# ) as dst:
#     dst.write(igbp, 1)

# dst.close()

# fs.put_file(local_target, remote_target)
# os.remove(local_target)

In [ ]:
igbp = xr.open_rasterio(
    "https://storage.googleapis.com/carbonplan-scratch/trace_scratch/IGBP.tif",
    parse_coordinates=True,
)
igbp = igbp.squeeze(dim="band", drop=True)
igbp = igbp.rename(x="lon", y="lat")
igbp

In [ ]:
igbp.plot()

In [ ]:
# let's just not reproject since the source file is so small. we'll read the data in directly

### EOSD


The following block copies the shapefiles from a ftp site.

There is an alternative source for EOSD data in tif format
http://tree.pfc.forestry.ca/ But would have needed untar unzip and to merge the
tif files together


In [ ]:
# import shutil
# import ftplib
# import urllib.request as request
# from contextlib import closing

# # source filepaths
# ftp_server = 'ftp.maps.canada.ca'
# path = '/pub/nrcan_rncan/vector/geobase_lcc_csc/shp_en/'
# # dest filepaths
# dest_path = 'carbonplan-scratch/trace_scratch/EOSD/'

# ftp = ftplib.FTP(ftp_server)
# ftp.login()
# ftp.cwd(path)
# folders = ftp.nlst()

# for folder in folders:
#     fnames = ftp.nlst(folder)
#     for fn in fnames:
#         fp = f'ftp://{ftp_server}{path}{fn}'
#         print(fp)
#         with closing(request.urlopen(fp)) as r:
#             uri = dest_path + fn
#             with fs.open(uri, 'wb') as f:
#                 shutil.copyfileobj(r, f)

The following blocks sorts the EOSD raw files according to the bounding box of
each tile


In [ ]:
from carbonplan_trace.v1.glas_allometric_eq import (
    parse_bounding_lat_lon_for_tile,
)
from shapely import geometry

In [ ]:
# get the list of tiles of interest and their respective bounding boxes
all_tiles = [
    p.split("/")[-1].split(".")[0]
    for p in fs.ls("gs://carbonplan-scratch/trace_scratch/ecoregions_mask/")
    if not p.endswith("/")
]

tile_poly = []
for tile in all_tiles:
    min_lat, max_lat, min_lon, max_lon = parse_bounding_lat_lon_for_tile(tile)
    tile_poly.append(
        geometry.Polygon(
            [
                [min_lon, min_lat],
                [min_lon, max_lat],
                [max_lon, max_lat],
                [max_lon, min_lat],
            ]
        )
    )

tile_gdf = gpd.GeoDataFrame(
    {"tile_name": all_tiles, "geometry": tile_poly}, crs="EPSG:4326"
)

In [ ]:
eosd_folder = "gs://carbonplan-scratch/trace_scratch/EOSD/"
eosd_subfolders = [p for p in fs.ls(eosd_folder) if not p.endswith("/")]

for eosd_subfolder in eosd_subfolders:
    zip_files = fs.ls(eosd_subfolder)
    for zip_file in zip_files:
        fn = zip_file.split("/")[-1]
        input_file = f"gs://{zip_file}"
        eosd_raw = gpd.read_file(input_file)
        min_lon, min_lat, max_lon, max_lat = eosd_raw.total_bounds
        eosd_poly = geometry.Polygon(
            [
                [min_lon, min_lat],
                [min_lon, max_lat],
                [max_lon, max_lat],
                [max_lon, min_lat],
            ]
        )
        # figure out which tile it belongs to
        intersect_tiles = tile_gdf.loc[
            tile_gdf.intersects(eosd_poly)
        ].tile_name.values
        for intersect_tile in intersect_tiles:
            fs.cp(
                f"gs://{zip_file}",
                f"gs://carbonplan-scratch/trace_scratch/EOSD_sorted/{intersect_tile}/{fn}",
            )

In [ ]:
mapper = fsspec.get_mapper(
    f"gs://carbonplan-scratch/trace_scratch/ecoregions_mask/50N_080W.zarr"
)
ds = xr.open_dataset(mapper, engine="zarr", cache=False)
ds

For each tile, concatenate everything and turn into raster


In [ ]:
all_tiles = [
    p
    for p in fs.ls(f"gs://carbonplan-scratch/trace_scratch/EOSD_sorted/")
    if not p.endswith("/")
]

for tile in all_tiles:
    fn = tile.split("/")[-1]
    tile_path = (
        f"gs://carbonplan-scratch/trace_scratch/ecoregions_mask/{fn}.zarr"
    )

    if fs.exists(tile_path + "/eosd/"):
        # if we have already process this tile, also pass
        print(f"Skipping {fn}, EOSD data already present")
        pass

    else:
        print(f"Processing {fn}")
        zip_files = fs.ls(tile)
        eosd = []
        for zf in zip_files:
            print(f"    reading {zf}")
            temp = gpd.read_file("gs://" + zf)
            print(temp.total_bounds)
            eosd.append(temp)

        print("concat")
        eosd = pd.concat(eosd, ignore_index=True)

        # read in the target tile
        target_tile = get_tile_in_xr(tile_path)

        eosd = eosd[["COVTYPE", "geometry"]]
        eosd = eosd.sort_values(by="COVTYPE").reset_index(drop=True)

        print("convert")
        # convert ecoregions shapefile into target tile format
        eosd_index_a = (
            convert_gdf_into_tiles(
                tile_ds=target_tile.isel(
                    lat=slice(0, 20000), lon=slice(0, 20000)
                ),
                gdf=eosd,
                value_col=None,
                value_name="eosd_index",
            )
            .chunk({"lat": 625, "lon": 1250})
            .to_dataset()
        )

        eosd_index_b = (
            convert_gdf_into_tiles(
                tile_ds=target_tile.isel(
                    lat=slice(20000, 40000), lon=slice(0, 20000)
                ),
                gdf=eosd,
                value_col=None,
                value_name="eosd_index",
            )
            .chunk({"lat": 625, "lon": 1250})
            .to_dataset()
        )

        eosd_index_c = (
            convert_gdf_into_tiles(
                tile_ds=target_tile.isel(
                    lat=slice(0, 20000), lon=slice(20000, 40000)
                ),
                gdf=eosd,
                value_col=None,
                value_name="eosd_index",
            )
            .chunk({"lat": 625, "lon": 1250})
            .to_dataset()
        )

        eosd_index_d = (
            convert_gdf_into_tiles(
                tile_ds=target_tile.isel(
                    lat=slice(20000, 40000), lon=slice(20000, 40000)
                ),
                gdf=eosd,
                value_col=None,
                value_name="eosd_index",
            )
            .chunk({"lat": 625, "lon": 1250})
            .to_dataset()
        )

        eosd_index = xr.combine_by_coords(
            [eosd_index_a, eosd_index_b, eosd_index_c, eosd_index_d]
        )["eosd_index"]

        print("nulls in total dataset", eosd_index.isnull().sum().values)

        print("get output dataset")
        eosd_cov = xr.DataArray(
            np.nan,
            dims=["lat", "lon"],
            coords=[target_tile.coords["lat"], target_tile.coords["lon"]],
        ).chunk({"lat": 625, "lon": 1250})

        print("assigning covers")
        covers = eosd.COVTYPE.unique()
        for c in covers:
            min_ind = np.where(eosd.COVTYPE == c)[0].min()
            max_ind = np.where(eosd.COVTYPE == c)[0].max()
            eosd_cov = xr.where(
                ((eosd_index >= min_ind) & (eosd_index <= max_ind)),
                x=c,
                y=eosd_cov,
            )

        print("put to output ds")
        target_tile["eosd"] = eosd_cov

        print("saving")
        # save the output
        save_to_zarr(
            ds=target_tile, url=tile_path, list_of_variables=["eosd"], mode="a"
        )

        del eosd_index
        del eosd_index_a
        del eosd_index_b
        del eosd_index_c
        del eosd_index_d
        del eosd_cov
        del target_tile
        del eosd

In [ ]:
mapper = fsspec.get_mapper(
    f"gs://carbonplan-scratch/trace_scratch/ecoregions_mask/70N_100W.zarr"
)
ds = xr.open_zarr(mapper)
ds

In [ ]:
ds.ecoregion.isnull().sum().values

In [ ]:
ds.ecoregion[::100, ::100].plot()

In [ ]:
ds.eosd.isnull().sum().values

In [ ]:
ds.eosd[::100, ::100].plot()